In [253]:
# Tarek El-Hajjaoui
import tensorflow as tf
import numpy as np
import pandas as pd
import string # needed to remove punctation from words
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, LSTM, Dropout, Activation, Embedding, Bidirectional
from sklearn.model_selection import train_test_split

In [197]:
def remove_punc(word, punc_set = string.punctuation):
  return ''.join(ch for ch in word if ch not in punc_set)

In [198]:
# lambda helper func
is_new_cat = lambda line : True if (len(line) != 0 and line[-1].isupper()) else False

def parse_txt_file_str(filename="/content/nursery_rhymes.txt"):
  parse_dict = {}
  curr_cat = ""
  with open(filename) as file:
    print(parse_dict)
    for line in file:
      strip_line = line.replace('\r', ' ').replace('\n', ' ').strip()
      if len(strip_line) == 0:
        continue
      new_category = is_new_cat(strip_line)
      if new_category:
        lower_cat = strip_line.lower().replace('_', ' ')
        if lower_cat not in parse_dict:
          parse_dict[lower_cat] = []
          curr_cat = lower_cat
      else:
        new_words = strip_line.split(' ')
        new_words = [word.lower() for word in new_words if word not in ['', ' ']]
        new_words = [remove_punc(word) for word in new_words]
        # assign new words to words list
        parse_dict[curr_cat] += new_words
  # returns dict with the key being the category of rhymes, values being
  # strings of rhymes concatenated to each other
  return parse_dict

In [325]:
parse_dict = parse_txt_file_str()

{}


In [326]:
def output_parse_data(file="parsed_data.txt", parse_dict=parse_dict):
  with open(file, "w") as out_file:
    out_file.write(str(parse_dict))

In [327]:
output_parse_data()

In [332]:
df_dict = {'rhyme_category': [], 'words': []}
words_lst = []
# key is rhyme category, value is list of words for that rhyme category
for key, value in parse_dict.items():
  df_dict['rhyme_category'].append(remove_punc(key))
  df_dict['words'].append((' '.join(value)))
  words_lst.append(value)

In [333]:
df_dict['rhyme_category'] = np.array(df_dict['rhyme_category'])
df_dict['words'] = np.array(df_dict['words'])

In [334]:
data_frame = pd.DataFrame(df_dict)

In [335]:
data_frame.head()

,rhyme_category,words
0,the queen of hearts,the queen of hearts she made some tarts all on...
1,saint swithins day,st swithins day if thou dost rain for forty da...
2,dance to your daddie,dance to your daddie my bonnie laddie dance to...
3,the man in the moon,the man in the moon came tumbling down and ask...
4,simple simon,simple simon met a pieman going to the fair sa...


In [336]:
data_frame.tail()

,rhyme_category,words
271,old chairs to mend,if id as much money as i could spend i never w...
272,old mother hubbard,mother hubbards old dog tray if this account b...
273,to babylon,how many miles is it to babylon threescore mil...
274,my black hen,hickety pickety my black hen she lays eggs for...
275,ill tell you a story,ill tell you a story about jack a nory and now...


In [337]:
data_frame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 276 entries, 0 to 275
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   rhyme_category  276 non-null    object
 1   words           276 non-null    object
dtypes: object(2)
memory usage: 4.4+ KB


In [338]:
def df_to_csv(file="rhyme_data.csv", data_frame=data_frame):
  data_frame.to_csv(file)

In [339]:
df_to_csv()

Set hyperparameters

In [340]:
num_categories = data_frame.shape[0]

In [296]:
vocab_size = 5000 # make the top list of words (common words)
embedding_dim = 276
max_length = 200
trunc_type = 'post'
padding_type = 'post'
oov_tok = '<OOV>' # OOV = Out of Vocabulary
training_portion = .8

In [297]:
type(df_dict['rhyme_category']), type(df_dict['words'])

(numpy.ndarray, numpy.ndarray)

In [309]:
X = df_dict['rhyme_category']
y = df_dict['words']
X.shape, y.shape

((276,), (276,))

In [344]:
# create corpus
total_words = []
for row in data_frame['words']:
  words = row.split(' ')
  for word in words:
    total_words.append(word)
len(total_words)

16030

In [322]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=training_portion, random_state=42)

DEPRECATED

In [146]:
# # lambda helper func
# is_new_cat = lambda line : True if (len(line) != 0 and line[-1].isupper()) else False

# def parse_txt_file_lst(filename="/content/nursery_rhymes.txt"):
#   parse_dict = {}
#   curr_cat = ""
#   with open(filename) as file:
#     print(parse_dict)
#     for line in file:
#       strip_line = line.replace('\r', ' ').replace('\n', ' ').strip()
#       if len(strip_line) == 0:
#         continue
#       new_category = is_new_cat(strip_line)
#       if new_category:
#         if strip_line not in parse_dict:
#           parse_dict[strip_line] = []
#           curr_cat = strip_line
#       else:
#         parse_dict[curr_cat].append(strip_line)
#   # returns dict with the key being the category of rhymes, values being
#   # list of rhymes
#   return parse_dict